Ref:
1. save numpy array
    https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
2. customize loss function
    https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618
3. Tensor Customize Loss Func
    https://www.tensorflow.org/guide/keras/train_and_evaluate#custom_losses

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import re
import glob
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
import cv2
import geopandas as gp
import descartes
from shapely.geometry import Polygon
from sklearn.preprocessing import MinMaxScaler

## Generating the boolean mask for each cell in the grid

In [3]:
# Grid_BoolMask
taiwan_grid = gp.read_file("./MapData/taiwan_grid.shp")
taiwan_offgrid = gp.read_file("./MapData/taiwan_offgrid.shp")
assert taiwan_grid.crs == 'epsg:3824' 
assert taiwan_offgrid.crs == 'epsg:3824' 

taiwan_grid['value'] = 0
taiwan_offgrid['value'] = 0
featmap = pd.concat([taiwan_grid, taiwan_offgrid], ignore_index=True)

dpi = 100
width_pixel = 200
height_pixel = 350

# generate boolmask for grids
for i in tqdm.tqdm_notebook(range(len(taiwan_grid))):
    featmap.iloc[i, 2] = 1
    fig, ax = plt.subplots(figsize=(width_pixel/dpi, height_pixel/dpi), dpi=dpi)
    canvas = FigureCanvasAgg(fig)

    featmap.plot(ax=ax, column='value', cmap='gray')

    ax.set_axis_off()
    ax.set_xlim(left=taiwan_offgrid.total_bounds[0], right=taiwan_offgrid.total_bounds[2])
    ax.set_ylim(bottom=taiwan_offgrid.total_bounds[1], top=taiwan_offgrid.total_bounds[3])
    #print(plt.xlim(), plt.ylim())
    #ax.margins(0)
    #fig.tight_layout(pad=0)

    # Retrieve a view on the renderer buffer
    canvas.draw()
    buf = canvas.buffer_rgba()
    # convert to a NumPy array
    X = np.asarray(buf)
    assert sum(sum(X[:,:,0] != X[:,:,1])) == 0
    assert sum(sum(X[:,:,2] != X[:,:,1])) == 0
    X = X[:,:,0]
    width_mean = X.mean(axis=0)
    height_mean = X.mean(axis=1)

    width_ind = np.arange(X.shape[-1])
    height_ind = np.arange(X.shape[0])

    left = width_ind[list(width_mean != 255.0)][0]
    right = width_ind[list(width_mean != 255.0)][-1]
    top = height_ind[list(height_mean != 255.0)][0]
    bottom = height_ind[list(height_mean != 255.0)][-1]

    img = X[top:bottom+1, left:right+1]

    bool_mask = (img == 255)
    #assert bool_mask.shape == img.shape
    np.save(f'./Grid_BoolMask/grid_{i}.npy', bool_mask)
    
    featmap.iloc[i, 2] = 0
    
    plt.close()

assert len(sorted(glob.glob('Grid_BoolMask/*.npy'))) == len(taiwan_grid)

## Generate the truthmap of sunshine hour and GloblRad from the main weather station 

In [4]:
# MainSta's location
taiwan_grid = gp.read_file("./MapData/taiwan_grid.shp")
taiwan_offgrid = gp.read_file("./MapData/taiwan_offgrid.shp")
Sta_df = pd.read_csv('WeatherStation.csv')
Sta_gdf = gp.GeoDataFrame(
    Sta_df, geometry=gp.points_from_xy(Sta_df.Longitude, Sta_df.Latitude), crs=taiwan_grid.crs)
MainSta_gdf = Sta_gdf.loc[0:8]

assert taiwan_grid.crs == 'epsg:3824' 
assert taiwan_offgrid.crs == 'epsg:3824' 

# locate the main stations in the grid
target = dict()
for i in range(len(MainSta_gdf)):
    for u in range(len(taiwan_grid)):
        if MainSta_gdf.loc[i, 'geometry'].within(taiwan_grid.loc[u, 'geometry']):
            target[i] = u
#target.pop('永康')
print(target)


# loop over the files, fill_CODiS
fill_csv = sorted(glob.glob('./fill_CODiS/fill2_Station_bytime_2017/*.csv'))


# time by time for MainSta's 'SunShine', 'GloblRad'
for name in tqdm.tqdm_notebook(fill_csv):
    
    npy_name = re.search(r'\d{4}-\d{2}-\d{2}_H\d{2}', name).group()
    
    # main station only
    codis = pd.read_csv(name, usecols=['SunShine', 'GloblRad']).iloc[:9] 
    #print(codis)

    for feat in ['SunShine', 'GloblRad']:
        truthmap = np.zeros((200, 155), dtype='float32')
        
        # fill the array with the corresponding cell values
        ## applying boolean mask to the array and fill in the values from the fill_CODiS
        for sta, grid in target.items():
            #print(sta,grid)
            truthmap[np.load(f'./Grid_BoolMask/grid_{grid}.npy')] = codis.loc[sta, feat]
        
        assert truthmap.shape == (200,155)
        assert truthmap.dtype == 'float32'
        np.save(f'./TruthMap/{feat}/{npy_name}.npy', truthmap)
        #plt.imshow(truthmap, cmap='gray')

{0: 8, 1: 8, 2: 11, 3: 12, 4: 37, 5: 44, 6: 36, 7: 65, 8: 59}


In [5]:
'''
img = np.load('./TruthMap/BoolMask.npy')
plt.imshow(img)
#plt.colorbar()
plt.savefig('./TruthMap/BoolMask.png')

bool_mask = (img != 0)
#print(bool_mask.shape)

#np.save('./TruthMap/BoolMask.npy', bool_mask)
'''

"\nimg = np.load('./TruthMap/BoolMask.npy')\nplt.imshow(img)\n#plt.colorbar()\nplt.savefig('./TruthMap/BoolMask.png')\n\nbool_mask = (img != 0)\n#print(bool_mask.shape)\n\n#np.save('./TruthMap/BoolMask.npy', bool_mask)\n"

In [ ]:
'''
# scale_mask
taiwan_offgrid = gp.read_file("./MapData/taiwan_offgrid.shp")
taiwan_grid = gp.read_file("./MapData/taiwan_grid.shp")
taiwan_grid['mask'] = 0.0
taiwan_offgrid['mask'] = 0.0
taiwan_offgrid.loc[71, 'mask'] = 1.0
scale_map = pd.concat([taiwan_grid, taiwan_offgrid], ignore_index=True)

# 100 pixel = 1" DPI
dpi = 100
width_pixel = 200
height_pixel = 350

fig, ax = plt.subplots(figsize=(width_pixel/dpi, height_pixel/dpi), dpi=dpi)
canvas = FigureCanvasAgg(fig)

scale_map.plot(ax=ax, column='mask', cmap='gray')

ax.set_axis_off()
ax.set_xlim(left=taiwan_offgrid.total_bounds[0], right=taiwan_offgrid.total_bounds[2])
ax.set_ylim(bottom=taiwan_offgrid.total_bounds[1], top=taiwan_offgrid.total_bounds[3])
#print(plt.xlim(), plt.ylim())
#ax.margins(0)
#fig.tight_layout(pad=0)

# Retrieve a view on the renderer buffer
canvas.draw()
buf = canvas.buffer_rgba()
# convert to a NumPy array
X = np.asarray(buf)
X = X[:,:,0]
width_mean = X.mean(axis=0)
height_mean = X.mean(axis=1)

width_ind = np.arange(X.shape[-1])
height_ind = np.arange(X.shape[0])

left = width_ind[list(width_mean != 255.0)][0]
right = width_ind[list(width_mean != 255.0)][-1]
top = height_ind[list(height_mean != 255.0)][0]
bottom = height_ind[list(height_mean != 255.0)][-1]

img = X[top:bottom+1, left:right+1]
#print(img.shape)
#print(img)
scale_mask = (img != 0)
#print(scale_mask.shape)
np.save('./TruthMap/ScaleMask.npy', scale_mask)

#plt.imshow(img)
#plt.savefig('./TruthMap/ScaleMask.png')
'''